In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,VotingRegressor,StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.manifold import TSNE
from catboost import CatBoostRegressor
from scipy import stats
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction
### The purpose of this notebook is to give some information and approach of using ensemble methods from custom estimators. I did not really apply a lot of Features Engineering and Transformation to save your time and ,indeed, show how stacking is powerful :)

In [ ]:
train = pd.read_csv("../input/bike-sharing-demand/train.csv", parse_dates=['datetime'])
test = pd.read_csv("../input/bike-sharing-demand/test.csv", parse_dates=['datetime'])

train.drop(['casual','registered'],axis=1,inplace=True)
train.drop(['temp'],axis=1,inplace=True)
test.drop(['temp'],axis=1,inplace=True)

print('train:',train.shape)
print('test:',test.shape)

### Here are just some features extraction from Timestamp

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek

train.drop(['datetime'],axis=1,inplace=True)
test_dates = test['datetime']

### Also, it is important to transform the target variable to avoid skewness and outliers

In [ ]:
sns.boxplot(train['count'])
plt.title('Boxplot of count without log transform')
plt.show()
train['count'] = train['count'].map(np.log1p)
sns.boxplot(train['count'])
plt.title('Boxplot of count with log transform')
plt.show()

# Modeling

In [ ]:
X = train.drop(['count'],axis=1)
y = train['count']

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

### So, First approach is voting. This is the simple one. Just average all predictions and give the overall answer. Anyway, with grid search it gave really good results, but Random Fores outperfomed voting. It seems like this data was constructed especcialy for him. Our goal is to build powerful stacking model, that's why let's try boosting methods next

In [ ]:
ridge = Ridge()
ridge_params = {'alpha':(0.001,0.005,0.01,0.05,0.1,0.5,1)}
grid = GridSearchCV(ridge,ridge_params,scoring='r2')
grid.fit(x_train,y_train)
ridge = grid.best_estimator_

svr = SVR()
svr_params = {'kernel':('linear', 'poly', 'rbf'),
             'C':(0.001,0.005)}
grid = GridSearchCV(svr,svr_params,scoring='r2')
grid.fit(x_train,y_train)
svr = grid.best_estimator_

kneighb = KNeighborsRegressor()
kneighb_params = {'n_neighbors': (range(2,15))}
grid = GridSearchCV(kneighb,kneighb_params,scoring='r2')
grid.fit(x_train,y_train)
kneighb = grid.best_estimator_

rf = RandomForestRegressor().fit(x_train,y_train)

voting_estimators = [('svr',svr),('rf',rf),('kneighb',kneighb),('ridge',ridge)]
voting = VotingRegressor(voting_estimators).fit(x_train,y_train)



In [ ]:
names = ['svr','kneighb','rf','ridge','voting']
scores = []
for counter,i in enumerate([svr,kneighb,rf,ridge,voting]):
    scores.append( mean_squared_error(y_test,i.predict(x_test)) )
    
tmp = pd.DataFrame(scores,names).T

plt.figure(figsize=(15,5))
plt.bar(names,scores,align='edge')

tmp.head(5)

### These models represents the best boosting libraries nowadays. Moreover, they do have amazing parametres. Like in previous chunks we will check the perfomance of each model separately before have a look at voting perfomance

In [ ]:
xgb = XGBRegressor(n_estimators=700,
                  max_depth=7,
                  learning_rate=0.05)
xgb.fit(x_train,y_train)

lgb = LGBMRegressor(n_estimators=600,
                   max_depth=6,
                   learning_rate=0.1)
lgb.fit(x_train,y_train)

ctb = CatBoostRegressor(n_estimators=500,
                   max_depth=5,
                   learning_rate=0.1,verbose=0)
ctb.fit(x_train,y_train)

gb_ensemble = [('xgb',xgb),('lgb',lgb),('ctb',ctb)]
voting_gb = VotingRegressor(gb_ensemble).fit(x_train,y_train)



In [ ]:
gb_ensemble = [('xgb',xgb),('lgb',lgb),('ctb',ctb)]
voting_gb = VotingRegressor(gb_ensemble).fit(x_train,y_train)

names_gb = ['xgb','lgb','ctb','gb ensemble']
scores_gb = []
for counter,i in enumerate([xgb,lgb,ctb,voting_gb]):
    scores_gb.append( mean_squared_error(y_test,i.predict(x_test)) )
    
tmp = pd.DataFrame(scores_gb,names_gb).T

tmp.head(5)

### Yes, Gradient Boosting Voting outperformed his 'weak' learners :)

### Now we will check each of models from the first chunk as a Meta Model in Stacking. By the way, usually for base learners are not boosting models, but in order to check something let's try. 

In [ ]:
stacked_estimators = [('xgb',xgb),('lgb',lgb),('ctb',ctb),]
finals = [svr,kneighb,rf,ridge,voting]

for counter,i in enumerate(finals):
    stacked = StackingRegressor(estimators = stacked_estimators,
                           final_estimator=i).fit(x_train,y_train)
    metric = mean_squared_error(y_test,stacked.predict(x_test))
    print(names[counter],' as meta-model score: ',metric)


### SVR model as a meta-model ouperfomed others slightly. But anyway, this is an amazing result.
### Upvote if you like my solution :)